# 4단계 YOLOv8 전이학습

## 4.1 YOLO에 입력 가능한 형태로 라벨 데이터 통합 및 변환
temp_label에 저장된 자세 정보를 읽어 원본 labels의 클래스 번호를 갱신한 뒤 maple-vision-search/data/processed/yolo_label에 저장
(temp_label과 raw/labels 데이터를 yolo 모델에 넣어줄 수 있는 형태로 통합)

In [8]:
import os
import re
import shutil
import yaml
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. 경로 설정
PROJECT_ROOT = os.path.expanduser("~/maple-vision-search")
RAW_IMG_DIR = os.path.join(PROJECT_ROOT, "data/raw/images")
RAW_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/raw/labels")
TEMP_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/temp_label")
YOLO_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/yolo_label")

# 학습용 최종 데이터셋 경로
YOLO_DATASET_ROOT = os.path.join(PROJECT_ROOT, "data/yolo_dataset")

for path in [YOLO_LABEL_DIR, YOLO_DATASET_ROOT]:
    os.makedirs(path, exist_ok=True)

# ---------------------------------------------------------
# STEP 1: 라벨 통합 (yolo_label 생성)
# ---------------------------------------------------------
mapping_table = {}
temp_files = [f for f in os.listdir(TEMP_LABEL_DIR) if f.endswith('.txt')]

for f in temp_files:
    # 정규식 수정: 파일명에 .png나 .jpg가 포함된 경우도 대응
    # 예: 'maple_0.png_0.txt' -> group(1): maple_0.png, group(2): 0
    match = re.search(r"(.+)_(\d+)\.txt$", f)
    if match:
        full_name = match.group(1)
        # 확장자 제거 (maple_0.png -> maple_0)
        img_basename = os.path.splitext(full_name)[0]
        obj_idx = int(match.group(2))
        
        with open(os.path.join(TEMP_LABEL_DIR, f), 'r') as file:
            pose_id = file.read().strip()
            
        if img_basename not in mapping_table:
            mapping_table[img_basename] = {}
        mapping_table[img_basename][obj_idx] = pose_id

print(f"매핑된 원본 이미지: {len(mapping_table)}개")

# 결합 실행
for img_name, objects in mapping_table.items():
    raw_path = os.path.join(RAW_LABEL_DIR, f"{img_name}.txt")
    if not os.path.exists(raw_path): continue
    
    with open(raw_path, 'r') as f:
        lines = f.readlines()
    
    final_lines = []
    for idx, line in enumerate(lines):
        parts = line.strip().split()
        if not parts: continue
        # 오토 라벨링된 포즈로 교체 (매핑 없으면 기존값 유지)
        parts[0] = objects.get(idx, parts[0])
        final_lines.append(" ".join(parts) + "\n")
    
    with open(os.path.join(YOLO_LABEL_DIR, f"{img_name}.txt"), 'w') as f:
        f.writelines(final_lines)

# ---------------------------------------------------------
# STEP 2: 데이터셋 분할 (Train/Val Split)
# ---------------------------------------------------------
# 먼저 전체에서 20%를 떼어내어 Val+Test용으로 만듦
train_imgs, temp_imgs = train_test_split(valid_images, test_size=0.2, random_state=42)

# 남은 20%를 다시 절반으로 나누어 Val(10%)과 Test(10%)로 분할
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

# 파일 이동 실행
move_files(train_imgs, 'train')
move_files(val_imgs, 'val')
move_files(test_imgs, 'test') # test 폴더 생성 및 복사 추가

# data.yaml에도 test 경로 추가
data_config['test'] = 'images/test'

# ---------------------------------------------------------
# STEP 3: data.yaml 생성
# ---------------------------------------------------------
data_config = {
    'path': YOLO_DATASET_ROOT,
    'train': 'images/train',
    'val': 'images/val',
    'nc': 10,
    'names': ['fly', 'jump', 'prone', 'shoot1', 'stabO1', 'stand1', 'stand2', 'swingO1', 'walk1', 'walk2']
}

with open(os.path.join(YOLO_DATASET_ROOT, 'data.yaml'), 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"데이터셋 준비 완료: {len(train_imgs)} train / {len(val_imgs)} val")

매핑된 원본 이미지: 489개
데이터셋 준비 완료: 391 train / 49 val


## 4.2 YOLOv8 모델 학습

In [9]:
from ultralytics import YOLO
import os

PROJECT_ROOT = os.path.expanduser("~/maple-vision-search")
data_yaml_path = os.path.join(PROJECT_ROOT, "data/yolo_dataset/data.yaml")

# 1. YOLOv8 모델 로드 (Pre-trained)
model = YOLO('yolov8n.pt') 

# 2. 전이 학습 (과적합 방지를 위해 Backbone 동결 전략 활용)
model.train(
    data=data_yaml_path,
    epochs=100,      # 데이터가 489개로 적으므로 충분히 학습
    imgsz=640,
    batch=16,
    freeze=10,       # Backbone 동결하여 과적합 방지
    project='maple_yolo',
    name='pose_detection_v1',
    device=0         # GPU 사용해서 학습
)

New https://pypi.org/project/ultralytics/8.3.243 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.241 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11901MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/ssy/maple-vision-search/data/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, n

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x73f958229960>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

## 4.3 학습 결과 시각화

1. 결과 이미지
![result.png](/home/ssy/maple-vision-search/notebook/maple_yolo/pose_detection_v1/results.png)


2. 혼동행렬 이미지
![confusion_matrix_normalized.png](/home/ssy/maple-vision-search/notebook/maple_yolo/pose_detection_v1/confusion_matrix_normalized.png)

3. 검증 데이터 라벨/예측 시각화

3.1 라벨

![val_batch0_labels.jpg](/home/ssy/maple-vision-search/notebook/maple_yolo/pose_detection_v1/val_batch0_labels.jpg)

3.2 예측

![val_batch0_pred.jpg](/home/ssy/maple-vision-search/notebook/maple_yolo/pose_detection_v1/val_batch0_pred.jpg)

## 4.4 평가 및 결과 시각화

`/home/ssy/maple-vision-search/data/yolo_dataset/images/test` 폴더의 데이터만 사용하여 평가(test)를 진행하고, 최종 성능 지표를 구한다.

In [ ]:
import pandas as pd

# 1. Test 데이터셋 평가 수행
print("--- [평가] Test 데이터셋 상세 지표 산출 시작 ---")
results = model.val(data=DATA_YAML, split='test', plots=True)

# 2. 전체 요약 지표 추출
mp, mr, map50, map95 = results.box.mean_results()
fitness = results.fitness

print("\n" + "="*50)
print(f"{'Overall Performance (Test Set)':^50}")
print("="*50)
print(f"- Mean Precision: {mp:.4f}")
print(f"- Mean Recall:    {mr:.4f}")
print(f"- mAP50:          {map50:.4f}")
print(f"- mAP50-95:       {map95:.4f}")
print(f"- Fitness:        {fitness:.4f}")
print("-" * 50)

# 3. 클래스별 상세 지표 정리
# results.box.ap50 : 각 클래스의 IoU 0.5일 때의 AP
# results.box.ap   : 각 클래스의 IoU 0.5:0.95일 때의 AP
stats = []
for i, name in results.names.items():
    class_map50 = results.box.ap50[i]
    class_map95 = results.box.ap[i]
    
    stats.append({
        "Class ID": i,
        "Class Name": name,
        "mAP50": round(float(class_map50), 4),
        "mAP50-95": round(float(class_map95), 4)
    })

df_stats = pd.DataFrame(stats)
print("\n[Per-Class Detailed Metrics]")
print(df_stats.to_string(index=False))

# 4. 추론 속도 확인
speed = results.speed
print("\n" + "-" * 50)
print(f"Speed: {speed['preprocess']:.2f}ms pre-process, "
      f"{speed['inference']:.2f}ms inference, "
      f"{speed['postprocess']:.2f}ms post-process per image")
print("=" * 50)

--- [평가] Test 데이터셋 상세 지표 산출 시작 ---
Ultralytics 8.3.241 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11901MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 5007.3±1942.3 MB/s, size: 304.5 KB)
val: Scanning /home/ssy/maple-vision-search/data/yolo_dataset/labels/test.cache... 49 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 49/49 145.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 9.5it/s 0.4s0.2s
                   all         49        147      0.746      0.811       0.86      0.755
                   fly         15         19      0.815      0.695      0.861      0.704
                  jump         25         29      0.478      0.759      0.716      0.591
                 prone         14         16      0.869       0.75      0.939      0.731
                shoot1         11         11      0.793      0.909      0.952      0.868
                stabO1          5          5   

## 4.5 테스트 결과 분석
1. mAP50 ($0.8600$) : 객체의 위치를 정확히 찾고 클래스를 분류할 확률이 86%
2. Mean Recall ($0.8109$) : 화면에 있는 캐릭터 10마리 중 약 8마리 이상을 놓치지 않고 찾아냄
3. Mean Precision ($0.7463$) : 모델이 '1번 포즈'라고 예측한 것 중 실제 1번 포즈일 확률이 75%이며, Recall에 비해 낮은 수치는 배경의 일부를 캐릭터로 오인하거나, 포즈를 잘못 판단한 경우가 종종 발생함을 시사
4. 추론 속도 (Total $\approx 5.36ms$) : Pre + Inference + Post 프로세스를 합쳐 장당 약 $5.4ms$가 소요. 이를 FPS로 환산하면 약 $186$ FPS.